# Crypto Data ETL Pipeline

Dieses Notebook ist für die Extraktion und Transformation der Kryptowährungsdaten zuständig.

In [1]:
import os
import requests
import pandas as pd
import json
from datetime import datetime
from dotenv import load_dotenv

## 1. Konfiguration

API-Schlüssel und Konstanten laden

In [2]:
load_dotenv()
API_KEY = os.getenv('COINMARKETCAP_API_KEY')
BASE_URL = 'https://pro-api.coinmarketcap.com/v1'
ENDPOINTS = {
    'listings': '/cryptocurrency/listings/latest'
}

## 2. Datenextraktion

Funktion zum Abrufen der Kryptowährungsdaten mit Fehlerbehandlung

In [3]:
def fetch_crypto_data(limit=10):
    """Kryptowährungsdaten von CoinMarketCap API abrufen"""
    try:
        url = BASE_URL + ENDPOINTS['listings']
        headers = {
            'Accepts': 'application/json',
            'X-CMC_PRO_API_KEY': API_KEY
        }
        params = {
            'start': '1',
            'limit': str(limit),
            'convert': 'USD'
        }
        
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
        
    except requests.exceptions.RequestException as e:
        print(f"Fehler beim API-Aufruf: {e}")
        return None

## 3. Datentransformation

Daten in ein strukturiertes Format umwandeln

In [4]:
def transform_crypto_data(raw_data):
    """Rohdaten in ein strukturiertes DataFrame umwandeln"""
    if not raw_data or 'data' not in raw_data:
        return None
        
    transformed_data = [{
        'Name': crypto['name'],
        'Symbol': crypto['symbol'],
        'Preis (USD)': round(crypto['quote']['USD']['price'], 2),
        'Marktkapitalisierung (USD)': round(crypto['quote']['USD']['market_cap'], 2),
        'Volumen 24h (USD)': round(crypto['quote']['USD']['volume_24h'], 2),
        'Änderung 24h (%)': round(crypto['quote']['USD']['percent_change_24h'], 2),
        'Zeitstempel': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    } for crypto in raw_data['data']]
    
    return pd.DataFrame(transformed_data)

## 4. Daten speichern

Transformierte Daten in verschiedenen Formaten speichern

In [5]:
def save_data(df, base_path='../data'):
    """Daten in verschiedenen Formaten speichern"""
    if df is None:
        return
        
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # CSV für historische Daten
    csv_path = os.path.join(base_path, f'crypto_data_{timestamp}.csv')
    df.to_csv(csv_path, index=False)
    
    # JSON für die Frontend-Visualisierung
    json_path = os.path.join(base_path, 'latest_crypto_data.json')
    df.to_json(json_path, orient='records')
    
    print(f"Daten gespeichert:\n- CSV: {csv_path}\n- JSON: {json_path}")

## 5. ETL-Pipeline ausführen

In [6]:
def run_etl_pipeline():
    """Gesamte ETL-Pipeline ausführen"""
    print("1. Daten extrahieren...")
    raw_data = fetch_crypto_data()
    
    if raw_data:
        print("2. Daten transformieren...")
        df = transform_crypto_data(raw_data)
        
        if df is not None:
            print("3. Daten speichern...")
            save_data(df)
            print("\nETL-Pipeline erfolgreich abgeschlossen!")
            return df
    
    print("Fehler in der ETL-Pipeline!")
    return None

# Pipeline ausführen
result_df = run_etl_pipeline()
if result_df is not None:
    display(result_df.head())

1. Daten extrahieren...
2. Daten transformieren...
3. Daten speichern...
Daten gespeichert:
- CSV: ../data/crypto_data_20241229_092718.csv
- JSON: ../data/latest_crypto_data.json

ETL-Pipeline erfolgreich abgeschlossen!


,Name,Symbol,Preis (USD),Marktkapitalisierung (USD),Volumen 24h (USD),Änderung 24h (%),Zeitstempel
0,Bitcoin,BTC,95001.54,1.881289e+12,2.295631e+10,0.87,2024-12-29 09:27:18
1,Ethereum,ETH,3395.62,4.090657e+11,1.334916e+10,1.94,2024-12-29 09:27:18
2,Tether USDt,USDT,1.00,1.387791e+11,6.505846e+10,0.01,2024-12-29 09:27:18
3,XRP,XRP,2.19,1.255286e+11,2.495899e+09,1.18,2024-12-29 09:27:18
4,BNB,BNB,715.34,1.030135e+11,1.918930e+09,2.78,2024-12-29 09:27:18
